In [1]:
import pandas as pd
import numpy as np
import os
import re 
import glob

In [2]:
def process_filename_info(df, filename):
    # Extract the base name (in case a full path is passed)
    base = os.path.basename(filename)
    
    # Regex to get race name and year
    match = re.match(r"f1_data_(.+)_([0-9]{4})\.csv", base)
    
    if match:
        race_name = match.group(1).replace("_", " ")  # Turn underscores into spaces
        year = int(match.group(2))
        
        df["RaceName"] = race_name
        df["Year"] = year
    else:
        print("⚠️ Filename doesn't match expected pattern.")
    
    return df

In [3]:
import pandas as pd

def impute_missing_practice_sessions(df):
    sessions = ["FP1", "FP2", "FP3"]
    penalty_seconds = 0.5

    # Explicitly initialize imputation flags
    for session in sessions:
        df[f"IsImputed_{session}"] = False

    for session in sessions:
        lap_col = f"LapTime_{session}"
        compound_col = f"LapCompound_{session}"

        missing_drivers = df[df[lap_col].isna()]

        for idx, driver_row in missing_drivers.iterrows():
            team, race, year, driver = (driver_row["TeamName"], 
                                        driver_row["RaceName"], 
                                        driver_row["Year"], 
                                        driver_row["Driver"])

            # Teammate's data for the session
            teammate = df[
                (df["TeamName"] == team) &
                (df["RaceName"] == race) &
                (df["Year"] == year) &
                (df["Driver"] != driver) &
                (~df[lap_col].isna())
            ]

            if not teammate.empty:
                # Use teammate's entire session data
                teammate_row = teammate.iloc[0]
                session_cols = [col for col in df.columns if f"_{session}" in col]
                df.loc[idx, session_cols] = teammate_row[session_cols].values
                df.at[idx, lap_col] += penalty_seconds

            else:
                # Impute LapTime as average
                avg_lap_time = df[
                    (df["RaceName"] == race) &
                    (df["Year"] == year) &
                    (~df[lap_col].isna())
                ][lap_col].mean()

                if pd.isna(avg_lap_time):
                    print(f"⚠️ Cannot impute average LapTime for {driver} in {session} at {race} ({year}).")
                    continue

                df.at[idx, lap_col] = avg_lap_time + penalty_seconds

                # Weather columns imputation as session average
                weather_cols = [col for col in df.columns if f"Weather_{session}" in col]
                for weather_col in weather_cols:
                    avg_weather = df[
                        (df["RaceName"] == race) &
                        (df["Year"] == year) &
                        (~df[weather_col].isna())
                    ][weather_col].mean()

                    df.at[idx, weather_col] = avg_weather

                # LapCompound set explicitly to "SOFT"
                if compound_col in df.columns:
                    df.at[idx, compound_col] = "SOFT"

                # Other session columns (like sectors) are set to NaN explicitly
                other_session_cols = [
                    col for col in df.columns if f"_{session}" in col 
                    and col not in [lap_col, compound_col] + weather_cols
                ]
                df.loc[idx, other_session_cols] = pd.NA

            # Explicitly mark as imputed
            df.at[idx, f"IsImputed_{session}"] = True

            # Update FastestPracticeTime
            df.at[idx, "FastestPracticeTime"] = df.loc[
                idx, ["LapTime_FP1", "LapTime_FP2", "LapTime_FP3"]
            ].min()

    return df


In [ ]:
def process_race_data(csv_path, output_path):
    df = pd.read_csv(csv_path)

    process_filename_info(df, csv_path)
    
    # For now keep also Nan in TotalRaceTime, as might use dataset to predict qualifying times
    if 'TotalRaceTime' not in  df.columns:
        print(f"⚠️ Column 'TotalRaceTime' missing in {csv_path}. Skipping file.")
        return  # skip further processing for this file

    # Convert all relevant time columns to seconds
    time_cols = [
        "LapTime_FP1", "LapTime_FP2", "LapTime_FP3",
        "AvgLapTime_LongestStint_FP1", "AvgLapTime_LongestStint_FP2", "AvgLapTime_LongestStint_FP3",
        "Delta_FirstLastLap_FP1", "Delta_FirstLastLap_FP2", "Delta_FirstLastLap_FP3",
        "Q1_FastestLap", "Q2_FastestLap", "Q3_FastestLap"
    ]
    for col in time_cols:
        df[col] = pd.to_timedelta(df[col]).dt.total_seconds()

    # Fastest laps
    df["FastestPracticeTime"] = df[["LapTime_FP1", "LapTime_FP2", "LapTime_FP3"]].min(axis=1)
    df["FastestQualifyingTime"] = df[["Q1_FastestLap", "Q2_FastestLap", "Q3_FastestLap"]].min(axis=1)

    # Map compounds to categorical
    compounds = {"SOFT": 0, "MEDIUM": 1, "HARD": 2, "INTERMEDIATE": 3, "WET": 4}

    # Find compound of fastest FP lap
    conditions = [
        df["LapTime_FP1"] == df["FastestPracticeTime"],
        df["LapTime_FP2"] == df["FastestPracticeTime"],
        df["LapTime_FP3"] == df["FastestPracticeTime"]
    ]
    choices = [df["Compound_FP1"], df["Compound_FP2"], df["Compound_FP3"]]
    df["FastestPracticeCompound"] = np.select(conditions, choices, default=np.nan)
    
    # Map compounds
    compound_cols = ["FastestPracticeCompound", "LongestStintCompound_FP1", "LongestStintCompound_FP2", "LongestStintCompound_FP3",
                    "Compound_FP1", "Compound_FP2", "Compound_FP3",
                     ]
    for col in compound_cols:
        df[col] = df[col].map(compounds)


    # Normalize all speed trap data vs. session slowest
    speed_cols = [
        "SpeedST_FP1", "SpeedFL_FP1", "SpeedI1_FP1", "SpeedI2_FP1",
        "SpeedST_FP2", "SpeedFL_FP2", "SpeedI1_FP2", "SpeedI2_FP2",
        "SpeedST_FP3", "SpeedFL_FP3", "SpeedI1_FP3", "SpeedI2_FP3",
        "Q1_TopSpeedST", "Q2_TopSpeedST", "Q3_TopSpeedST"
    ]

    for col in speed_cols:
        df[col] = df[col] - df[col].min()


    # Normalize race time
    df["TotalRaceTime"] = pd.to_timedelta(df["TotalRaceTime"]).dt.total_seconds()
    totalTimeFirst = df["TotalRaceTime"].max()
    df["TotalRaceTime"] = df["TotalRaceTime"].fillna(totalTimeFirst)
    df.loc[df["TotalRaceTime"] != totalTimeFirst, "TotalRaceTime"] += totalTimeFirst

    df = impute_missing_practice_sessions(df)

    # Append to combined CSV (create it if doesn't exist)
    if not os.path.exists(output_path):
        df.to_csv(output_path, index=False)
    else:
        df.to_csv(output_path, mode='a', index=False, header=False)  # Append without writing header again

    print(f"Processed and saved: {csv_path}")


In [5]:
# Folder path (relative to script location)
data_folder = './data'

# Get all CSV files in the folder
csv_files = glob.glob(os.path.join(data_folder, '*.csv'))

# Loop through each CSV
for csv_file in csv_files:
    process_race_data(csv_file, "dataset/f1_data_combined.csv") 

Processed and saved: ./data/f1_data_Saudi_Arabian_Grand_Prix_2021.csv
Processed and saved: ./data/f1_data_Canadian_Grand_Prix_2023.csv
Processed and saved: ./data/f1_data_Monaco_Grand_Prix_2021.csv
Processed and saved: ./data/f1_data_Canadian_Grand_Prix_2022.csv
Processed and saved: ./data/f1_data_Abu_Dhabi_Grand_Prix_2018.csv


/var/folders/ty/jl1vm4217zx6vktvp7bd66nm0000gn/T/ipykernel_70772/4229074061.py:73: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.loc[idx, other_session_cols] = pd.NA
/var/folders/ty/jl1vm4217zx6vktvp7bd66nm0000gn/T/ipykernel_70772/4229074061.py:73: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.loc[idx, other_session_cols] = pd.NA
/var/folders/ty/jl1vm4217zx6vktvp7bd66nm0000gn/T/ipykernel_70772/4229074061.py:73: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.lo

Processed and saved: ./data/f1_data_Abu_Dhabi_Grand_Prix_2024.csv
Processed and saved: ./data/f1_data_Austria_Grand_Prix_2020.csv
Processed and saved: ./data/f1_data_Saudi_Arabian_Grand_Prix_2022.csv
Processed and saved: ./data/f1_data_Italian_Grand_Prix_2019.csv
Processed and saved: ./data/f1_data_British_Grand_Prix_2019.csv
Processed and saved: ./data/f1_data_Monaco_Grand_Prix_2022.csv
Processed and saved: ./data/f1_data_Bahrain_Grand_Prix_2018.csv
Processed and saved: ./data/f1_data_Bahrain_Grand_Prix_2024.csv


/var/folders/ty/jl1vm4217zx6vktvp7bd66nm0000gn/T/ipykernel_70772/4229074061.py:73: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.loc[idx, other_session_cols] = pd.NA
/var/folders/ty/jl1vm4217zx6vktvp7bd66nm0000gn/T/ipykernel_70772/4229074061.py:73: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.loc[idx, other_session_cols] = pd.NA
/var/folders/ty/jl1vm4217zx6vktvp7bd66nm0000gn/T/ipykernel_70772/4229074061.py:73: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.lo

Processed and saved: ./data/f1_data_Bahrain_Grand_Prix_2025.csv
Processed and saved: ./data/f1_data_Bahrain_Grand_Prix_2019.csv
Processed and saved: ./data/f1_data_British_Grand_Prix_2024.csv
Processed and saved: ./data/f1_data_Monaco_Grand_Prix_2023.csv
Processed and saved: ./data/f1_data_British_Grand_Prix_2018.csv
Processed and saved: ./data/f1_data_Italian_Grand_Prix_2018.csv
Processed and saved: ./data/f1_data_Italian_Grand_Prix_2024.csv
Processed and saved: ./data/f1_data_Saudi_Arabian_Grand_Prix_2023.csv
Processed and saved: ./data/f1_data_Abu_Dhabi_Grand_Prix_2019.csv


...

Processed and saved: ./data/f1_data_Mexican_Grand_Prix_2022.csv
Processed and saved: ./data/f1_data_Mexican_Grand_Prix_2023.csv
Processed and saved: ./data/f1_data_Azerbaijan_Grand_Prix_2022.csv
Processed and saved: ./data/f1_data_Qatar_Grand_Prix_2021.csv
Processed and saved: ./data/f1_data_Japanese_Grand_Prix_2024.csv
Processed and saved: ./data/f1_data_Japanese_Grand_Prix_2018.csv
⚠️ Column 'TotalRaceTime' missing in ./data/f1_data_Japanese_Grand_Prix_2019.csv. Skipping file.


/var/folders/ty/jl1vm4217zx6vktvp7bd66nm0000gn/T/ipykernel_70772/4229074061.py:73: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.loc[idx, other_session_cols] = pd.NA
/var/folders/ty/jl1vm4217zx6vktvp7bd66nm0000gn/T/ipykernel_70772/4229074061.py:73: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.loc[idx, other_session_cols] = pd.NA
/var/folders/ty/jl1vm4217zx6vktvp7bd66nm0000gn/T/ipykernel_70772/4229074061.py:73: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.lo

Processed and saved: ./data/f1_data_Japanese_Grand_Prix_2025.csv
Processed and saved: ./data/f1_data_Sakhir_Grand_Prix_2020.csv
Processed and saved: ./data/f1_data_Azerbaijan_Grand_Prix_2019.csv
Processed and saved: ./data/f1_data_Mexican_Grand_Prix_2018.csv


/var/folders/ty/jl1vm4217zx6vktvp7bd66nm0000gn/T/ipykernel_70772/4229074061.py:73: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.loc[idx, other_session_cols] = pd.NA
/var/folders/ty/jl1vm4217zx6vktvp7bd66nm0000gn/T/ipykernel_70772/4229074061.py:73: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.loc[idx, other_session_cols] = pd.NA
/var/folders/ty/jl1vm4217zx6vktvp7bd66nm0000gn/T/ipykernel_70772/4229074061.py:73: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  df.lo

Processed and saved: ./data/f1_data_Mexican_Grand_Prix_2024.csv
Processed and saved: ./data/f1_data_Brazilian_Grand_Prix_2018.csv
Processed and saved: ./data/f1_data_Russian_Grand_Prix_2020.csv
Processed and saved: ./data/f1_data_Brazilian_Grand_Prix_2019.csv
Processed and saved: ./data/f1_data_Mexican_Grand_Prix_2019.csv
Processed and saved: ./data/f1_data_Azerbaijan_Grand_Prix_2018.csv
Processed and saved: ./data/f1_data_Azerbaijan_Grand_Prix_2024.csv
